## Documents Extraction and Processing

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Lets first obtain the file that maps the tickers with the CIKs

In [2]:
import requests
import json
from pathlib import Path
import os

# --- Configuration (from previous step) ---
HEADERS = {
    "User-Agent": "EdgarTutorial/1.0 (YourName your.email@domain.com)" 
}
TICKER_CIK_URL = "https://www.sec.gov/files/company_tickers.json"
OUTPUT_FILE = Path("sec_data/company_tickers.json")

# Ensure directory exists
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
# ----------------------------------------

# 1. Download the JSON data
print("Downloading CIK-Ticker map...")
response = requests.get(TICKER_CIK_URL, headers=HEADERS, timeout=15)
response.raise_for_status()
raw_data = response.json() # Load into Python dictionary

# 2. Open the file and use json.dump() with indent=4
print(f"Saving JSON in readable format to {OUTPUT_FILE.absolute()}...")

# Use 'w' mode to write the file
with open(OUTPUT_FILE, 'w') as f:
    # Key Fix: The 'indent=4' parameter tells the JSON module to format the output 
    # with 4 spaces for each level of nesting, adding line breaks automatically.
    json.dump(raw_data, f, indent=4) 

print("✅ JSON saved successfully with proper line breaks and indentation.")

# --- Optional: Print a Snippet to Console (Also Pretty-Printed) ---
# If you want to print to the console instead of a file, use json.dumps()
print("\n--- Console Snippet (Pretty-Printed) ---")
# Print the first 3 key-value pairs from the dictionary
keys = list(raw_data.keys())
snippet = {k: raw_data[k] for k in keys[:3]}

# Use json.dumps() with indent=2 to format the string output
pretty_string = json.dumps(snippet, indent=2)
print(pretty_string)

Saving JSON in readable format to /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json...
✅ JSON saved successfully with proper line breaks and indentation.

--- Console Snippet (Pretty-Printed) ---
{
  "0": {
    "cik_str": 1045810,
    "ticker": "NVDA",
    "title": "NVIDIA CORP"
  },
  "1": {
    "cik_str": 1652044,
    "ticker": "GOOGL",
    "title": "Alphabet Inc."
  },
  "2": {
    "cik_str": 320193,
    "ticker": "AAPL",
    "title": "Apple Inc."
  }
}


### 10K Company filings

In [8]:
from edgar import set_identity, Company

# 1. Set your identity (Required by SEC: "Name email@domain.com")
set_identity("luis.alvarez.conde@alumnos.upm.es")

# 2. Get the company (Apple)
ticker = "AAPL"
company = Company(ticker)

# 1. Get recent Form 4 filings (Insider Trading)
insider_filings = company.get_filings(form="4").head(20)

people_nodes = {}

for f in insider_filings:
    # Parse the Form 4 into an object
    form4 = f.obj()
    
    # Extract Reporting Owners (The Executives)
    for owner in form4.reporting_owners:
        name = owner.name
        title = owner.officer_title # e.g., "Chief Executive Officer"
        
        # Deduplicate
        if name not in people_nodes:
            people_nodes[name] = title

# Output for Graph Construction
for name, title in people_nodes.items():
    print(f"👤 Found Person: {name} | Role: {title}")
    # Create Relation: (Person {name: name}) -[:HAS_ROLE {title: title}]-> (Company {ticker: 'AAPL'})

    
# 3. Get the latest 10-K filing
# We filter by form "10-K" and get the single latest result
latest_10k = company.get_filings(form="10-K").latest()

# 4. Get the URL of the primary document
# 'homepage_url' is the index page; 'document.url' is the actual HTML/Text file
filing_url = latest_10k.document.url

# 5. Extract the text content
# The library automatically cleans HTML tags when you use .text()
content = latest_10k.text()


tenk = latest_10k.obj()
business_text = tenk['Item 1']
print(business_text)
print(f"Type: {type(tenk)}")
print("\nProperties/Attributes:")
for attr in dir(tenk):
    if not attr.startswith('_'):
        print(f"  {attr}")
# 6. Save content to a text file
filename = f"{ticker}_10K_latest.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(content)

# 7. Output results
print(f"✅ Successfully saved 10-K content to {filename}")
print(f"🔗 Link to official filing: {filing_url}")

👤 Found Person: Katherine L. Adams | Role: SVP, GC and Secretary
👤 Found Person: Chris Kondo | Role: Principal Accounting Officer
👤 Found Person: Kevan Parekh | Role: Senior Vice President, CFO
👤 Found Person: Timothy D Cook | Role: Chief Executive Officer
👤 Found Person: Deirdre O'Brien | Role: Senior Vice President
👤 Found Person: Sabih Khan | Role: COO
👤 Found Person: Arthur D Levinson | Role: None
Item 1.    Business

Company Background

The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September.

Products

iPhone

iPhone® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 17 Pro, iPhone Air™, iPhone 17, iPhone 16 and iPhone 16e.

Mac

Mac® is the Company’s line of personal computers based on its macOS® operating system. The Mac line inc

### Vanguard Index Funds

In [3]:
import pandas as pd
from io import StringIO
from edgar import Company, set_identity 
import sys
from pathlib import Path

RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

from src.simple_rag.extraction.parser import BlackRockFiling


set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "VOO"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    
    latest_date_str = max(f.report_date for f in all_filings)
    
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    print("Found filings: ", len(latest_filings), "for year: ", target_year)


funds_total = []
performance_funds = []
df_performance = []
for filing in latest_filings:

    print("Processing filing: ", filing.report_date)
    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    for fund in funds:
        if fund.performance_table is not None:
            performance_funds.append(fund.ticker)
            count += 1

    df_performance.append(parser.get_financial_highlights())

    print(count)
    print("Adding funds: ", len(funds))
    
    funds_total.extend(funds)

print(len(performance_funds))
print(performance_funds)
print(len(df_performance))


Found filings:  2 for year:  2024
Processing filing:  2024-12-31
Processing: Vanguard Extended Market Index Fund
Extracting context:  FY2024_C000007779Member
Tag not found:  dei:SecurityExchangeName FY2024_C000007779Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
No tables found for block:  oef:LineGraphTableTextBlock


ValidationError: 1 validation error for FundData
provider
  Field required [type=missing, input_value={'name': 'Vanguard Extend...  9.33%  
4    12.48%  }, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [ ]:
parser.print_fund_info(funds_total)\
    

Showing information of 52 funds


### 🏦 Vanguard Extended Market Index Fund

🆔 Context ID:      FY2024_C000007779Member
🎫 Ticker:          VEXMX
🏷️ Share Class:     Investor Shares
📅 Report Date:     December 31, 2024
🏛️ Sec Exchange:    N/A

--- 💰 Costs & Financials ---
Net Assets          : 111,156
Expense Ratio       : 0.19
Turnover Rate       : 11
Costs per $10k      : 21
Advisory Fees       : 1,799
Number of Holdings  : 3,485

📝 Commentary: "How did the Fund perform during the reporting period?   For the 12 months ended December 31, 2024, the Fund performed roughly in line with its benchmark index.   U.S. economic growth hovered around 3% on a year-over-year basis for much of the period,..."


**📊 Average Annual Returns**

,0,1,2,3
0,Average Annual Total Returns,NaN,NaN,NaN
1,NaN,1 Year,5 Years,10 Years
2,Investor Shares,16.76%,9.75%,9.31%
3,S&P Completion Index,16.88%,9.77%,9.33%
4,Dow Jones U.S. Total Stock Market Float Adjust...,23.88%,13.78%,12.48%


...

,Unnamed: 0,Institutional Shares,CRSP US Large Cap Index,Dow Jones U.S. Total Stock Market Float Adjusted Index
0,2014,"$5,000,000","$5,000,000","$5,000,000"
1,2015,"$5,064,717","$5,065,516","$5,090,054"
2,2015,"$5,072,527","$5,073,505","$5,096,078"
3,2015,"$4,733,990","$4,735,147","$4,725,720"
4,2015,"$5,053,400","$5,055,301","$5,022,045"
5,2016,"$5,103,277","$5,108,683","$5,067,957"
6,2016,"$5,229,161","$5,235,336","$5,200,452"
7,2016,"$5,440,308","$5,446,752","$5,431,090"
8,2016,"$5,642,490","$5,649,147","$5,655,796"
9,2017,"$5,991,191","$5,998,908","$5,983,419"


**📊 Average Annual Returns**

,Unnamed: 0,1 Year,5 Years,10 Years
0,Institutional Shares,25.12%,14.49%,13.02%
1,CRSP US Large Cap Index,25.15%,14.51%,13.05%
2,Dow Jones U.S. Total Stock Market Float Adjust...,23.88%,13.78%,12.48%


**🏗️ Sector Allocation**

,0,1,2
0,Basic Materials,1.2%,NaN
1,Consumer Discretionary,15.1%,NaN
2,Consumer Staples,4.0%,NaN
3,Energy,3.2%,NaN
4,Financials,10.8%,NaN
5,Health Care,9.8%,NaN
6,Industrials,11.3%,NaN
7,Real Estate,2.0%,NaN
8,Technology,38.0%,NaN
9,Telecommunications,2.0%,NaN


In [ ]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1]], ignore_index=True)

returns_lookup = total_df.copy()

display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']
for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:        
            returns_lookup[f'{col}_clean'] = (
                returns_lookup[col]
                .astype(str)
                .str.replace('%', '')
                .str.replace('$', '')
                .str.replace(',', '')
                .replace('N/A', '0')
                .replace('', '0')
                .replace('None', '0')
                .astype(float)
            )
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)

        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Available share classes: {name_matches['share_class'].unique()}")
        continue
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        
       
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",len(funds_total))
    

2


array(['Small-Cap Index Fund', 'Small-Cap Growth Index Fund',
       'Small-Cap Value Index Fund', 'Extended Market Index Fund',
       'Mid-Cap Index Fund', 'Mid-Cap Growth Index Fund',
       'Mid-Cap Value Index Fund', 'Total Stock Market Index Fund',
       '500 Index Fund', 'Growth Index Fund', 'Value Index Fund',
       'Large-Cap Index Fund'], dtype=object)


Processing fund object: Vanguard Extended Market Index Fund - Investor Shares
Cleaned name: 'Extended Market Index Fund'
  Found 30 name matches
  Found 5 matching records
  2024: Total Return = 16.76%, Expense Ratio = 0.19%, Net Assets = 195.0, Net Income Ratio = 1.09, Turnover = 11.0, Net Assets Value Begining = 124.78, Net Assets Value End = 144.2
  2023: Total Return = 25.22%, Expense Ratio = 0.19%, Net Assets = 232.0, Net Income Ratio = 1.28, Turnover = 11.0, Net Assets Value Begining = 100.93, Net Assets Value End = 124.78
  2022: Total Return = -26.56%, Expense Ratio = 0.19%, Net Assets = 229.0, Net Income Ratio = 1.14, Turnover = 11.0, Net Assets Value Begining = 138.8, Net Assets Value End = 100.93
  2021: Total Return = 12.31%, Expense Ratio = 0.19%, Net Assets = 399.0, Net Income Ratio = 0.87, Turnover = 19.0, Net Assets Value Begining = 124.83, Net Assets Value End = 138.8
  2020: Total Return = 32.04%, Expense Ratio = 0.19%, Net Assets = 454.0, Net Income Ratio = 1.04, Tu

In [ ]:
import sys
%reload_ext autoreload
from src.simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
  2015 Return: $10,000.00 -> $10,125.00 = 1.25%
  2016 Return: $10,125.00 -> $11,321.00 = 11.81%
  2017 Return: $11,321.00 -> $13,774.00 = 21.67%
  2018 Return: $13,774.00 -> $13,151.00 = -4.52%
  2019 Return: $13,151.00 -> $17,271.00 = 31.33%
  2020 Return: $17,271.00 -> $20,423.00 = 18.25%
  2021 Return: $20,423.00 -> $26,250.00 = 28.53%
  2022 Return: $26,250.00 -> $21,465.00 = -18.23%
  2023 Return: $21,465.00 -> $27,069.00 = 26.11%
  2024 Return: $27,069.00 -> $33,794.00 = 24.84%

Final Annual Returns:
  VFINX: {'2015': 1.25, '2016': 11.81, '2017': 21.67, '2018': -4.52, '2019': 31.33, '2020': 18.25, '2021': 28.53, '2022': -18.23, '2023': 26.11, '2024': 24.84}
dict_keys(['2024', '2023', '2022', '2021', '2020'])
    2015: turnover=0.0 expense_ratio=0.0 total_return=1.

/home/alvar/CascadeProjects/windsurf-project/RAG/src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')


## Vanguard World Fund

In [ ]:
import pandas as pd
from io import StringIO
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))
%reload_ext autoreload
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company


set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "MGK"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    # 1. Find the most recent date in the entire history (e.g., "2024-12-31")
    latest_date_str = max(f.report_date for f in all_filings)
    
    # 2. Extract just the YEAR (e.g., "2024")
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    target_year = "2024"
    filings2 = sorted(
        [f for f in all_filings if f.report_date and f.report_date.startswith(target_year)],
        key=lambda f: f.report_date,
        reverse=True
    )

    latest_filings.append(filings2[0])
    print("Found filings: ", len(latest_filings), "for year: ", target_year)



performance_funds = []
df_performance = []
world_funds = set()

abort = False
for filing in latest_filings:

    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    for fund in funds:
        if fund.performance_table is not None:
            if fund.ticker not in performance_funds:
                performance_funds.append(fund.ticker)
                count += 1
        if fund.ticker not in world_funds:
            world_funds.add(fund.ticker)
        else:
            print("Exiting filing, repeated ticker found: ", fund.ticker)
            abort = True
            break
    if not abort:

        df_performance.append(parser.get_financial_highlights())
        print(count)
        print("Adding funds: ", len(funds))
        funds_total.extend(funds)

print("Total world funds added: ", len(world_funds))
print(len(performance_funds))
print(performance_funds)

print(len(df_performance))


Found filings:  4 for year:  2024
Processing: Mega Cap Growth Index Fund
Extracting context:  From2024-10-01to2025-09-30_C000055216Member
Processing: Mega Cap Growth Index Fund
Extracting context:  From2024-10-01to2025-09-30_C000055215Member
Tag not found:  dei:SecurityExchangeName From2024-10-01to2025-09-30_C000055215Member
2
Adding funds:  2
Processing: Vanguard Extended Duration Treasury Index Fund
Extracting context:  FY2025_C000051981Member
Tag not found:  dei:SecurityExchangeName FY2025_C000051981Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
No tables found for block:  oef:LineGraphTableTextBlock
Processing: Vanguard Extended Duration Treasury Index Fund
Extracting context:  FY2025_C000051979Member
Tag not found:  dei:SecurityExchangeName FY2025_C000051979Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
No tables found for block:  oef:LineGraphTableTextBlock
Processing: Vanguard ESG U.S. Stock ETF
Extracting context:  FY2025_C000

In [ ]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1], df_performance[2]], ignore_index=True)

returns_lookup = total_df.copy()

# Optional: Clean the total_return column (remove % sign if needed)
print(returns_lookup.head())
display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']


for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:  
            try:      
                returns_lookup[f'{col}_clean'] = (
                    returns_lookup[col]
                    .astype(str)
                    .str.replace('%', '')
                    .str.replace('$', '')
                    .str.replace(',', '')
                    .replace('N/A', '0')
                    .replace('', '0')
                    .replace('None', '0')
                    .astype(float)
                )
            except Exception as e:
                print(f"Error cleaning column '{col}': {str(e)}")
                print(returns_lookup[col].to_string())
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "™" in name:
        name = name.replace("™", "")
    elif "®" in name:
        name = name.replace("®", "")
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Available share classes: {name_matches['share_class'].unique()}")
        continue
    elif len(share_class_matches) > 5:
        print("  More than 5 share class matches found:")
        print(share_class_matches)
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        
       
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",world_funds)
    

3
                    fund_name share_class  year  net_assets  nav_beginning  \
0  Mega Cap Growth Index Fund  ETF Shares  2025     31195.0         321.87   
1  Mega Cap Growth Index Fund  ETF Shares  2024     22954.0         314.83   
2  Mega Cap Growth Index Fund  ETF Shares  2024     21996.0         241.25   
3  Mega Cap Growth Index Fund  ETF Shares  2023     14376.0         195.20   
4  Mega Cap Growth Index Fund  ETF Shares  2022     11168.0         248.50   

   nav_end  total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0   402.45         25.58           0.07              0.42                14.0   
1   321.87          2.35           0.07              0.40                 6.0   
2   314.83         31.16           0.07              0.51                14.0   
3   241.25         24.39           0.07              0.62                 7.0   
4   195.20        -21.08           0.07              0.51                 5.0   

  distribution_shares  
0                N

array(['Mega Cap Growth Index Fund',
       'Extended Duration Treasury Index Fund', 'ESG U.S. Stock ETF',
       'ESG International Stock ETF', 'Global Wellington Fund',
       'Global Wellesley Income Fund', 'ESG U.S. Corporate Bond ETF',
       'U.S. Growth Fund', 'International Growth Fund',
       'FTSE Social Index Fund', 'Communication Services Index Fund',
       'Consumer Discretionary Index Fund', 'Consumer Staples Index Fund',
       'Energy Index Fund', 'Financials Index Fund',
       'Health Care Index Fund', 'Industrials Index Fund',
       'Information Technology Index Fund', 'Materials Index Fund',
       'Utilities Index Fund', 'Mega Cap Index Fund',
       'Mega Cap Value Index Fund'], dtype=object)


Processing fund object: Vanguard Extended Market Index Fund - Investor Shares
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VEXMX

Processing fund object: Vanguard Extended Market Index Fund - ETF Shares
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VXF

Processing fund object: Vanguard Extended Market Index Fund - Admiral Shares
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VEXAX

Processing fund object: Vanguard Extended Market Index Fund - Institutional Shares
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VIEIX

Processing fund object: Vanguard Extended Market Index Fund - Institutional Plus Shares
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VEMPX

Processing fund object: Vanguard Extended Market Index Fund - Institutional Select Shares
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker

In [ ]:
import sys
%reload_ext autoreload
sys.path.append('../src')


from simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2016 Return: $10,724.00 -> $11,409.00 = 6.39%
  2017 Return: $11,409.00 -> $14,772.00 = 29.48%
  2018 Return: $14,772.00 -> $14,349.00 = -2.86%
  2019 Return: $14,349.00 -> $19,736.00 = 37.54%
  2020 Return: $19,736.00 -> $27,826.00 = 40.99%
  2021 Return: $27,826.00 -> $35,753.00 = 28.49%
  2022 Return: $35,753.00 -> $23,755.00 = -33.56%
  2023 Return: $23,755.00 -> $36,004.00 = 51.56%
  2024 Return: $36,004.00 -> $47,873.00 = 32.97%
  2025 Return: $47,873.00 -> $56,289.00 = 17.58%

Final Annual Returns:
  MGK: {'2016': 6.39, '2017': 29.48, '2018': -2.86, '2019': 37.54, '2020': 40.99, '2021': 28.49, '2022': -33.56, '2023': 51.56, '2024': 32.97, '2025': 17.58}
dict_keys(['2025', '2024', '2023', '2022', '2021'])
    2016: turnover=0.0 expense_ratio=0.0 total_return=6.39

In [ ]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Successfully saved 93 funds to pickle file
File size: 395.12 KB


## Vanguard Specialized Funds

In [ ]:
import pandas as pd
from io import StringIO

import sys
from pathlib import Path
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company


set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "VDIGX"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    # 1. Find the most recent date in the entire history (e.g., "2024-12-31")
    latest_date_str = max(f.report_date for f in all_filings)
    
    # 2. Extract just the YEAR (e.g., "2024")
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    
    print("Found filings: ", len(latest_filings), "for year: ", target_year)



performance_funds = []
specialized_funds = set()
df_performance = []
abort = False
for filing in latest_filings:

    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    for fund in funds:
        if fund.performance_table is not None:
            if fund.ticker not in performance_funds: 
                performance_funds.append(fund.ticker)
                count += 1
        if fund.ticker not in specialized_funds:
            specialized_funds.add(fund.ticker)
        else:
            print("Exiting filing, repeated ticker found: ", fund.ticker)
            abort = True
            break
    if abort:
        break
    df_performance.append(parser.get_financial_highlights())

    print(count)
    print("Adding funds: ", len(funds))
    funds_total.extend(funds)

print("Specialized funds: ", len(specialized_funds))
print(len(performance_funds))
print(performance_funds)
print(len(df_performance))



Found filings:  2 for year:  2025
Processing: Dividend Growth Fund
Extracting context:  From2024-02-01to2025-01-31_C000008004Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008004Member
Processing: Energy Fund
Extracting context:  From2024-02-01to2025-01-31_C000008005Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008005Member
Processing: Energy Fund
Extracting context:  From2024-02-01to2025-01-31_C000008006Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008006Member
Processing: Health Care Fund
Extracting context:  From2024-02-01to2025-01-31_C000008007Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008007Member
Processing: Health Care Fund
Extracting context:  From2024-02-01to2025-01-31_C000008008Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008008Member
Processing: Dividend Appreciation Index Fund
Extracting context:  From2024-02

In [ ]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1]], ignore_index=True)

returns_lookup = total_df.copy()

# Optional: Clean the total_return column (remove % sign if needed)
print(returns_lookup.head())
display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']


for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:  
            try:      
                returns_lookup[f'{col}_clean'] = (
                    returns_lookup[col]
                    .astype(str)
                    .str.replace('%', '')
                    .str.replace('$', '')
                    .str.replace(',', '')
                    .replace('N/A', '0')
                    .replace('', '0')
                    .replace('None', '0')
                    .astype(float)
                )
            except Exception as e:
                print(f"Error cleaning column '{col}': {str(e)}")
                print(returns_lookup[col].to_string())
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    if fund_obj.ticker not in specialized_funds:
        continue
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "™" in name:
        name = name.replace("™", "")
    elif "®" in name:
        name = name.replace("®", "")
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Available share classes: {name_matches}")
        print(f"  Found {len(share_class_matches)} matching records")
        count += 1
        # Add all matching returns
        for _, row in share_class_matches.iterrows():
            year = str(row['year'])
            highlights = FinancialHighlights(
                turnover=row.get('portfolio_turnover_clean', 0),
                expense_ratio=row.get('expense_ratio_clean', 0),
                total_return=row['total_return'],
                net_assets=row.get('net_assets_clean', 0),
                net_assets_value_begining=row.get('nav_beginning_clean', 0),
                net_assets_value_end=row.get('nav_end_clean', 0),
                net_income_ratio=row.get('net_income_ratio_clean', 0.0)
            )
            
            fund_obj.financial_highlights[year] = highlights
            print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
        continue
    elif len(share_class_matches) > 5:
        print("  More than 5 share class matches found:")
        print(share_class_matches)
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",specialized_funds)
    

2
              fund_name share_class  year  net_assets  nav_beginning  nav_end  \
0  Dividend Growth Fund        None  2025     50424.0          37.76    37.14   
1  Dividend Growth Fund        None  2024     52553.0          35.42    37.76   
2  Dividend Growth Fund        None  2023     53452.0          37.85    35.42   
3  Dividend Growth Fund        None  2022     54186.0          31.82    37.85   
4  Dividend Growth Fund        None  2021     45099.0          30.63    31.82   

   total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0         10.20           0.22              1.68                16.0   
1          9.11           0.29              1.74                 9.0   
2         -0.76           0.30              1.68                11.0   
3         25.66           0.27              1.56                15.0   
4          7.03           0.26              1.85                15.0   

  distribution_shares  
0                None  
1                None  
2     

array(['Dividend Growth Fund', 'Energy Fund', 'Health Care Fund',
       'Dividend Appreciation Index Fund', 'Real Estate Index Fund',
       'Real Estate II Index Fund', 'Global Capital Cycles Fund',
       'Global ESG Select Stock Fund'], dtype=object)


Processing fund object: Dividend Growth Fund - Investor Shares
Cleaned name: 'Dividend Growth Fund'
  Found 5 name matches
  No share class matches found for 'Investor Shares' ticker:  VDIGX
  Available share classes:               fund_name share_class  year  net_assets  nav_beginning  nav_end  \
0  Dividend Growth Fund        None  2025     50424.0          37.76    37.14   
1  Dividend Growth Fund        None  2024     52553.0          35.42    37.76   
2  Dividend Growth Fund        None  2023     53452.0          37.85    35.42   
3  Dividend Growth Fund        None  2022     54186.0          31.82    37.85   
4  Dividend Growth Fund        None  2021     45099.0          30.63    31.82   

   total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0         10.20           0.22              1.68                16.0   
1          9.11           0.29              1.74                 9.0   
2         -0.76           0.30              1.68                11.0   
3     

In [ ]:
import sys
%reload_ext autoreload
sys.path.append('../src')


from simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2016 Return: $10,619.00 -> $10,874.00 = 2.40%
  2017 Return: $10,874.00 -> $12,860.00 = 18.26%
  2018 Return: $12,860.00 -> $14,143.00 = 9.98%
  2019 Return: $14,143.00 -> $16,879.00 = 19.35%
  2020 Return: $16,879.00 -> $17,545.00 = 3.95%
  2021 Return: $17,545.00 -> $23,646.00 = 34.77%
  2022 Return: $23,646.00 -> $22,953.00 = -2.93%
  2023 Return: $22,953.00 -> $23,134.00 = 0.79%
  2024 Return: $23,134.00 -> $28,121.00 = 21.56%
  2025 Return: $28,121.00 -> $28,555.00 = 1.54%

Final Annual Returns:
  VDIGX: {'2016': 2.4, '2017': 18.26, '2018': 9.98, '2019': 19.35, '2020': 3.95, '2021': 34.77, '2022': -2.93, '2023': 0.79, '2024': 21.56, '2025': 1.54}
dict_keys([])
    2016: turnover=0.0 expense_ratio=0.0 total_return=2.4 net_assets=0.0 net_assets_value_begining=0.0 ne

In [ ]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Successfully saved 107 funds to pickle file
File size: 451.74 KB


In [ ]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 107 funds from pickle file


## Vanguard Whitehall Funds

In [ ]:
import pandas as pd
from io import StringIO
import sys
from pathlib import Path
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company


set_identity("luis.alvarez.conde@alumnos.upm.es")
ticker = "VMGRX"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    # 1. Find the most recent date in the entire history (e.g., "2024-12-31")
    latest_date_str = max(f.report_date for f in all_filings)
    
    # 2. Extract just the YEAR (e.g., "2024")
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    
    print("Found filings: ", len(latest_filings), "for year: ", target_year)



performance_funds = []
whitehall_funds = set()
df_performance = []
abort = False

for filing in latest_filings:

    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    for fund in funds:
        if fund.performance_table is not None:
            if fund.ticker not in performance_funds: 
                performance_funds.append(fund.ticker)
                count += 1

        if fund.ticker not in whitehall_funds:
            whitehall_funds.add(fund.ticker)
        else:
            print("Exiting filing, repeated ticker found: ", fund.ticker)
            abort = True
            break
    if abort:
        break

    df_performance.append(parser.get_financial_highlights())

    print(count)
    print("Adding funds: ", len(funds))
    funds_total.extend(funds)

print("Whitehall funds: ", len(whitehall_funds))
print(len(performance_funds))
print(whitehall_funds)
print(len(df_performance))


Found filings:  2 for year:  2025
Processing: Mid-Cap Growth Fund
Extracting context:  From2024-11-01to2025-10-31_C000012166Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000012166Member
Processing: Selected Value Fund
Extracting context:  From2024-11-01to2025-10-31_C000012167Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000012167Member
Processing: Emerging Markets Government Bond Index Fund
Extracting context:  From2024-11-01to2025-10-31_C000126408Member
Processing: Emerging Markets Government Bond Index Fund
Extracting context:  From2024-11-01to2025-10-31_C000126407Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000126407Member
Processing: Emerging Markets Government Bond Index Fund
Extracting context:  From2024-11-01to2025-10-31_C000126409Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000126409Member
Processing: Global Minimum Volatility Fund
Extracting context:  From2

In [ ]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1]], ignore_index=True)

returns_lookup = total_df.copy()

# Optional: Clean the total_return column (remove % sign if needed)
print(returns_lookup.head())
returns_lookup['fund_name'] = (
    returns_lookup['fund_name']
    .str.replace('\n', ' ', regex=False)
)
display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']


for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:  
            try:      
                returns_lookup[f'{col}_clean'] = (
                    returns_lookup[col]
                    .astype(str)
                    .str.replace('%', '')
                    .str.replace('$', '')
                    .str.replace(',', '')
                    .replace('N/A', '0')
                    .replace('', '0')
                    .replace('None', '0')
                    .astype(float)
                )
            except Exception as e:
                print(f"Error cleaning column '{col}': {str(e)}")
                print(returns_lookup[col].to_string())
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    if fund_obj.ticker not in whitehall_funds:
        continue
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "™" in name:
        name = name.replace("™", "")
    elif "®" in name:
        name = name.replace("®", "")
    elif "\n" in name:
        name = name.replace("\n", " ")
        print(name)
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Found {len(name_matches)} name records")
        count += 1
        # Add all matching returns
        for _, row in name_matches.iterrows():
            year = str(row['year'])
            highlights = FinancialHighlights(
                turnover=row.get('portfolio_turnover_clean', 0),
                expense_ratio=row.get('expense_ratio_clean', 0),
                total_return=row['total_return'],
                net_assets=row.get('net_assets_clean', 0),
                net_assets_value_begining=row.get('nav_beginning_clean', 0),
                net_assets_value_end=row.get('nav_end_clean', 0),
                net_income_ratio=row.get('net_income_ratio_clean', 0.0)
            )
            
            fund_obj.financial_highlights[year] = highlights
            print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
        continue
    elif len(share_class_matches) > 5:
        print("  More than 5 share class matches found:")
        print(share_class_matches)
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",whitehall_funds)
    

2
             fund_name share_class  year  net_assets  nav_beginning  nav_end  \
0  Mid-Cap Growth Fund        None  2025      3116.0          26.21    29.55   
1  Mid-Cap Growth Fund        None  2024      3042.0          19.38    26.21   
2  Mid-Cap Growth Fund        None  2023      2530.0          19.24    19.38   
3  Mid-Cap Growth Fund        None  2022      2956.0          38.72    19.24   
4  Mid-Cap Growth Fund        None  2021      5290.0          29.89    38.72   

   total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0         14.77           0.32              0.26              1285.0   
1         35.77           0.33              0.37                69.0   
2          0.99           0.37              0.37                87.0   
3        -32.22           0.35              0.14                71.0   
4         37.68           0.33             -0.04                98.0   

  distribution_shares  
0                None  
1                None  
2           

array(['Mid-Cap Growth Fund', 'Selected Value Fund',
       'Emerging Markets Government Bond Index Fund',
       'Global Minimum Volatility Fund',
       'International Dividend Appreciation Index Fund',
       'International High Dividend Yield Index Fund',
       'International Dividend Growth Fund',
       'Advice Select International Growth Fund',
       'Advice Select Dividend Growth Fund',
       'Advice Select Global Value Fund', 'International Explorer Fund',
       'High Dividend Yield Index Fund'], dtype=object)


Processing fund object: Mid-Cap Growth Fund - Investor Shares
Cleaned name: 'Mid-Cap Growth Fund'
  Found 5 name matches
  No share class matches found for 'Investor Shares' ticker:  VMGRX
  Found 5 name records
  2025: Total Return = 14.77%, Expense Ratio = 0.32%, Net Assets = 3116.0, Net Income Ratio = 0.26, Turnover = 1285.0, Net Assets Value Begining = 26.21, Net Assets Value End = 29.55
  2024: Total Return = 35.77%, Expense Ratio = 0.33%, Net Assets = 3042.0, Net Income Ratio = 0.37, Turnover = 69.0, Net Assets Value Begining = 19.38, Net Assets Value End = 26.21
  2023: Total Return = 0.99%, Expense Ratio = 0.37%, Net Assets = 2530.0, Net Income Ratio = 0.37, Turnover = 87.0, Net Assets Value Begining = 19.24, Net Assets Value End = 19.38
  2022: Total Return = -32.22%, Expense Ratio = 0.35%, Net Assets = 2956.0, Net Income Ratio = 0.14, Turnover = 71.0, Net Assets Value Begining = 38.72, Net Assets Value End = 19.24
  2021: Total Return = 37.68%, Expense Ratio = 0.33%, Net Ass

In [ ]:
import sys
%reload_ext autoreload
sys.path.append('../src')


from simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2016 Return: $10,000.00 -> $9,451.00 = -5.49%
  2017 Return: $9,451.00 -> $11,595.00 = 22.69%
  2018 Return: $11,595.00 -> $12,710.00 = 9.62%
  2019 Return: $12,710.00 -> $14,434.00 = 13.56%
  2020 Return: $14,434.00 -> $17,079.00 = 18.32%
  2021 Return: $17,079.00 -> $23,515.00 = 37.68%
  2022 Return: $23,515.00 -> $15,939.00 = -32.22%
  2023 Return: $15,939.00 -> $16,097.00 = 0.99%
  2024 Return: $16,097.00 -> $21,856.00 = 35.78%
  2025 Return: $21,856.00 -> $25,084.00 = 14.77%

Final Annual Returns:
  VMGRX: {'2016': -5.49, '2017': 22.69, '2018': 9.62, '2019': 13.56, '2020': 18.32, '2021': 37.68, '2022': -32.22, '2023': 0.99, '2024': 35.78, '2025': 14.77}
dict_keys(['2025', '2024', '2023', '2022', '2021'])
    2016: turnover=0.0 expense_ratio=0.0 total_return=-5.49 

Found year-end data for years: [np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2017 Return: $10,964.00 -> $13,177.00 = 20.18%
  2018 Return: $13,177.00 -> $12,249.00 = -7.04%
  2019 Return: $12,249.00 -> $14,572.00 = 18.96%
  2020 Return: $14,572.00 -> $15,116.00 = 3.73%
  2021 Return: $15,116.00 -> $19,640.00 = 29.93%
  2022 Return: $19,640.00 -> $15,143.00 = -22.90%
  2023 Return: $15,143.00 -> $16,799.00 = 10.94%
  2024 Return: $16,799.00 -> $20,576.00 = 22.48%
  2025 Return: $20,576.00 -> $22,440.00 = 9.06%

Final Annual Returns:
  VIGI: {'2017': 20.18, '2018': -7.04, '2019': 18.96, '2020': 3.73, '2021': 29.93, '2022': -22.9, '2023': 10.94, '2024': 22.48, '2025': 9.06}
dict_keys(['2025', '2024', '2023', '2022', '2021'])
    2017: turnover=0.0 expense_ratio=0.0 total_return=20.18 net_assets=0.0 net_assets_value_begining=0.0 net_assets_value_end=0.0 net_income_ratio=0.0


/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/../src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')
/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/../src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')
/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/../src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['

In [ ]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Successfully saved 125 funds to pickle file
File size: 530.06 KB


In [ ]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/src")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 125 funds from pickle file


## Ishares

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd
from typing import List
import sys
from pathlib import Path
from tqdm import tqdm
%reload_ext autoreload
from simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company

set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "HEZU"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")

def process_single_filing_multiprocess(filing_data):
    """
    Process a single filing (for multiprocessing).
    Note: Must pass serializable data, not the filing object directly
    """
    try:
        # Import inside function for multiprocessing
        import sys
        from pathlib import Path
        sys.path.append('../src')
        from simple_rag.extraction.parser import BlackRockFiling
        
        html_content, report_date = filing_data
        parser = BlackRockFiling(html_content)
        funds = parser.get_funds()
        
        performance_funds = []
        df_performance = None        
        count = 0
        for fund in funds:

            if fund.performance_table is not None:
                if fund.ticker not in performance_funds:
                    performance_funds.append(fund.ticker)
                    count += 1
            if fund.ticker not in ishares_funds:
                ishares_funds.append(fund.ticker)
        
            print("Calling get_financial_highlights2")
        df_performance = parser.get_financial_highlights2()
        
        print(f"Filing {report_date}: Found {count} funds with performance tables, Total funds: {len(funds)}")
        
        return {
            'funds': funds,
            'performance_tickers': performance_funds,
            'df_performance': df_performance,
            'report_date': report_date
        }
    except Exception as e:
        print(f"Error processing filing: {e}")
        return None

if all_filings:
    unique_dates = sorted({f.report_date for f in all_filings if f.report_date})
    print("Unique report dates:", unique_dates)
    
    # Filter for filings from 2024-08-31 onward
    cutoff_date = "2024-09-31"
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date >= cutoff_date
    ]
    
    print("Found filings: ", len(latest_filings), "from", cutoff_date, "onward")
    
    # Optional: Show the dates of filtered filings
    print("Filtered filing dates:", sorted({f.report_date for f in latest_filings}))

# Prepare data for multiprocessing (fetch HTML first)
filing_data_list = []
failed_filings = []
for filing in latest_filings:
    try:
        html_content = filing.html()
        if html_content:  # Only add if HTML content exists
            filing_data_list.append((html_content, filing.report_date))
        else:
            print(f"⚠️  No HTML content for filing: {filing.report_date}")
            failed_filings.append(filing)
    except ValueError as e:
        print(f"❌ Error processing filing {filing.report_date}: {e}")
        failed_filings.append(filing)
    except Exception as e:
        print(f"❌ Unexpected error for filing {filing.report_date}: {e}")
        failed_filings.append(filing)
print(f"✅ Successfully prepared {len(filing_data_list)} filings")
print(f"❌ Failed to prepare {len(failed_filings)} filings")
# Continue with successful filings only

performance_funds = []
df_performances = []
ishares_funds = []

# Use ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    future_to_data = {executor.submit(process_single_filing_multiprocess, data): data 
                      for data in filing_data_list}
    
    for future in tqdm(as_completed(future_to_data), total=len(filing_data_list), desc="Processing filings"):
        result = future.result()
        if result:
            ishares_funds.extend(result['funds'])
            funds_total.extend(result['funds'])
            performance_funds.extend(result['performance_tickers'])
            
            if result['df_performance'] is not None:
                df_performances.append(result['df_performance'])

print(len(df_performances))
print(f"Total funds processed: {len(ishares_funds)}")


Unique report dates: ['2003-04-30', '2003-07-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-07-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-07-31', '2006-02-28', '2006-03-31', '2006-04-30', '2006-07-31', '2007-02-28', '2007-03-31', '2007-04-30', '2007-07-31', '2008-02-29', '2008-03-31', '2008-04-30', '2008-07-31', '2009-02-28', '2009-03-31', '2009-04-30', '2009-07-31', '2009-08-31', '2010-02-28', '2010-03-31', '2010-04-30', '2010-07-31', '2010-08-31', '2011-02-28', '2011-03-31', '2011-04-30', '2011-07-31', '2011-08-31', '2011-10-31', '2012-02-29', '2012-03-31', '2012-04-30', '2012-07-31', '2012-08-31', '2012-10-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-07-31', '2013-08-31', '2013-10-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-07-31', '2014-08-31', '2014-10-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-07-31', '2015-08-31', '2015-10-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-07-31', '2016-08-31', '2016-10-31', '2017-02-28', '2017-03-31'

Processing filings:   0%|          | 0/20 [00:00<?, ?it/s]

Processing: iShares iBonds 1-5 Year Corporate Ladder ETF
Extracting context:  From2024-11-01to2025-10-31_C000254885Member
Tag not found:  oef:ClassName From2024-11-01to2025-10-31_C000254885Member
Unknown Table:        Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Unknown table type:       Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Processing: iShares iBonds 1-5 Year High Yield and Income Ladder ETF
Extracting context:  From2024-11-01to2025-10-31_C000254886Member
Tag not found:  oef:ClassName From2024-11-01to2025-10-31_C000254886Member
Unknown Table:        Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Unknown table type:       Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Processing: iShares iBonds 1-5 Year TIPS Ladder ETF
Extracting context:  From2024-11-01to2025-10-31_C000254887Member
Tag not found:  oef:ClassName From2024-11-01to2

Processing filings:   5%|▌         | 1/20 [00:01<00:37,  1.95s/it]

      Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Unknown table type:       Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Processing: iShares Currency Hedged MSCI Eurozone ETF
Extracting context:  FY2025_C000141929Member
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Tag not found:  dei:SecurityExchangeName FY2025_C000141929Member
Found 3 potential Financial Highlights sections
Total funds extracted: 0Unknown Table: 
 Filing 2025-10-31: Found 0 funds with performance tables, Total funds: 4
      0                                                  1
0  ​(a)  The underlying fund is iShares MSCI Eurozone ETF.
1  ​(b)                       Excludes money market funds.
Unknown table type:       0                                                  1
0  ​(a)  The underlying fund is iShares MSCI Eurozone ETF.
1  ​(b)                

...

No data obtained
No data obtained
Processing: iShares iBonds Dec 2044 Term Treasury ETF
Extracting context:  FY2024_C000250197Member
No data obtained
Processing: iShares iBonds Dec 2034 Term Treasury ETF
Extracting context:  FY2024_C000250196Member
Tag not found:  oef:FactorsAffectingPerfTextBlock FY2024_C000250197Member
Unknown Table:        0                                                  1
0  ​(a)                       Excludes money market funds.
1    ​*  Credit quality ratings shown reflect the ratin...
Unknown table type:       0                                                  1
0  ​(a)                       Excludes money market funds.
1    ​*  Credit quality ratings shown reflect the ratin...
Tag not found:  dei:SecurityExchangeName FY2024_C000250196Member
Tag not found:  oef:FactorsAffectingPerfTextBlock FY2024_C000250196Member
No data obtained
Unknown Table:        0                                                  1
0  ​(a)                       Excludes money market fund

Processing filings:  95%|█████████▌| 19/20 [03:17<00:25, 25.16s/it]

Failed to extract tables from block:  oef:LineGraphTableTextBlock
No tables found for block:  oef:LineGraphTableTextBlock
Failed to extract tables from block:  oef:AvgAnnlRtrTableTextBlock
No tables found for block:  oef:AvgAnnlRtrTableTextBlock
Processing: iShares S&P 500 BuyWrite ETF
Extracting context:  FY2024_C000247832Member
Tag not found:  dei:SecurityExchangeName FY2024_C000247832Member
Unknown Table:        0                                                 1
0  ​(a)  The underlying fund is iShares Core S&P 500 ETF.
1  ​(b)                      Excludes money market funds.
Unknown table type:       0                                                 1
0  ​(a)  The underlying fund is iShares Core S&P 500 ETF.
1  ​(b)                      Excludes money market funds.
Failed to extract tables from block:  oef:LineGraphTableTextBlock
No tables found for block:  oef:LineGraphTableTextBlock
Failed to extract tables from block:  oef:AvgAnnlRtrTableTextBlock
No tables found for block:  oe

Processing filings: 100%|██████████| 20/20 [03:29<00:00, 10.48s/it]


18
Total funds processed: 380


In [ ]:
failed_filings[0].url

'https://www.sec.gov/Archives/edgar/data/1100663/0001100663-25-000026-index.html'

In [ ]:
import pandas as pd
import re
%reload_ext autoreload
from simple_rag.models.fund import FinancialHighlights

if df_performances:
    df_performance = pd.concat(df_performances, ignore_index=True)
else:
    df_performance = pd.DataFrame() # Empty fallback
    print("No performance data found.")

print(df_performance.head())

def clean_financial_number(val):
    """
    Parses financial strings like '23.19 %(b)' or '(24.82 )%'.
    - Extracts the numerical value.
    - Handles (12.34) as negative -12.34.
    - Ignores footnote markers like (a), (b).
    - Removes %, $, and commas.
    """
    if pd.isna(val) or val is None:
        return None
    
    # Convert to string and strip whitespace
    s = str(val).strip()
    
    # 1. Regex to find the number (handles decimals and commas)
    # Looks for digits, optional commas, and optional decimal part
    match = re.search(r'(\d{1,3}(?:,\d{3})*\.?\d*|\d*\.?\d+)', s)
    
    if not match:
        return None
        
    # Get the raw number string (e.g., "24.82" or "1,234.56")
    num_str = match.group(0)
    
    # 2. Check for negative indication: "(" at the start of the string
    # Accounting format always puts the negative parenthesis at the start: (24.82)%
    is_negative = s.startswith('(')
    
    try:
        # Remove commas and convert to float
        clean_num = float(num_str.replace(',', ''))
        
        # Apply negative sign if detected
        return -clean_num if is_negative else clean_num
        
    except ValueError:
        return None
returns_lookup = df_performance.copy()


# Apply to all financial columns
financial_cols = ['total_return', 'expense_ratio', 'net_income_ratio', 'portfolio_turnover', 'nav_end', 'nav_beginning', 'net_assets']
for col in financial_cols:
    if col in returns_lookup.columns:
        returns_lookup[f'{col}_clean'] = returns_lookup[col].apply(clean_financial_number)

ishares_tickers = [fund_obj.ticker for fund_obj in ishares_funds]
print("Tickers in ishares_funds:", ishares_tickers)


# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    
    if fund_obj.ticker not in ishares_tickers:
        continue

    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}

    if not hasattr(fund_obj, 'financial_highlights') or fund_obj.financial_highlights is None:
        fund_obj.financial_highlights = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "®" in name:
        name = name.replace("®", "")
    if "™" in name:
        name = name.replace("™", "")
        
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.contains(name, case=False, na=False, regex=False)]
    
    if len(name_matches) == 0:
        print("  No name matches found")
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    fund_obj.share_class = "ETF Shares"
    share_class = fund_obj.share_class
    
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if name_matches['share_class'].isna().all():
        fund_obj.annual_returns = dict(zip(name_matches['year'], name_matches['total_return_clean']))
        print("Annual return: ", fund_obj.annual_returns)
        continue
        
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}'")
        print(f"  Available share classes: {name_matches['share_class'].unique()}")
        continue
    
    print(f"  Found {len(share_class_matches)} matching records")
    
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        
        # Store annual return
        fund_obj.annual_returns[year] = row['total_return_clean']
        
        # Store full financial highlights snapshot
        fund_obj.financial_highlights[year] = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean'),
            expense_ratio=row.get('expense_ratio_clean'),
            total_return=row.get('total_return_clean'),
            net_assets=row.get('net_assets'),  # You may need to add this cleaning
            net_assets_value_begining=row.get('nav_beginning_clean'),
            net_assets_value_end=row.get('nav_end_clean') ,
            net_income_ratio=row.get('net_income_ratio_clean')
        )
    
    print(f"  Annual returns: {fund_obj.annual_returns}")
    print(f"  Financial highlights years: {list(fund_obj.financial_highlights.keys())}")
    for key, value in fund_obj.financial_highlights.items():
        print(f"    {key}: {value}")

                                   fund_name share_class  year   net_assets  \
0  iShares Large Cap Accelerated Outcome ETF  ETF Shares  2025          0.0   
1  iShares Large Cap Accelerated Outcome ETF  ETF Shares  2025   13473000.0   
2       iShares Large Cap Max Buffer Mar ETF  ETF Shares  2025   38203000.0   
3       iShares Large Cap Max Buffer Jun ETF  ETF Shares  2025  161134000.0   
4       iShares Large Cap Max Buffer Jun ETF  ETF Shares  2024   72687000.0   

   nav_beginning  nav_end  total_return  expense_ratio  net_income_ratio  \
0           0.00     0.00          0.00           0.00              0.00   
1          25.00    25.91          3.64           0.47              0.69   
2          25.00    25.81          3.25           0.47              0.65   
3          25.24    27.59         10.21           0.47              0.96   
4          25.00    25.24          0.95           0.47              0.00   

   portfolio_turnover distribution_shares  
0                 0.0   

In [ ]:
import re
from collections import defaultdict
import pandas as pd

def infer_first_col_format(value: object) -> str:
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return "EMPTY"

    s = str(value).strip()
    if s == "" or s.lower() == "nan":
        return "EMPTY"

    # Jan 23, Aug 15
    if re.match(r"^[A-Za-z]{3}\s+\d{2}$", s):
        return "MON_YY"

    # 2015
    if re.match(r"^\d{4}$", s):
        return "YYYY"

    # 2024-08-31
    if re.match(r"^\d{4}-\d{2}-\d{2}$", s):
        return "YYYY_MM_DD"

    # 08/31/24 or 8/31/2024
    if re.match(r"^\d{1,2}/\d{1,2}/\d{2,4}$", s):
        return "MM_DD_YY(YY)"

    # 31/08/24 (if you ever have EU style)
    if re.match(r"^\d{1,2}-\d{1,2}-\d{2,4}$", s):
        return "DD_MM_YY(YY)_or_MM_DD_YY(YY)_DASH"

    # Fallbacks
    if re.search(r"\d", s):
        return "OTHER_HAS_DIGITS"

    return "OTHER_TEXT"


def describe_first_column_formats(
    dfs,
    names=None,
    samples_per_df=3,
    max_groups_to_show=50,
    max_dfs_per_group_to_print=5,
):
    if names is None:
        names = [f"df[{i}]" for i in range(len(dfs))]

    groups = defaultdict(list)

    for name, df in zip(names, dfs):
        if df is None or not isinstance(df, pd.DataFrame) or df.empty:
            groups["EMPTY_DF"].append((name, df))
            continue

        first_col = df.columns[0]
        # take first non-empty sample from first column
        series = df[first_col].astype(str)
        sample_vals = [v for v in series.head(20).tolist() if str(v).strip() and str(v).lower() != "nan"]

        fmt = infer_first_col_format(sample_vals[0]) if sample_vals else "EMPTY_FIRST_COL"
        groups[fmt].append((name, df))

    sorted_groups = sorted(groups.items(), key=lambda kv: len(kv[1]), reverse=True)

    print(f"Total dataframes: {len(dfs)}")
    print(f"Unique first-column formats: {len(sorted_groups)}\n")

    for gi, (fmt, members) in enumerate(sorted_groups[:max_groups_to_show], start=1):
        print("=" * 100)
        print(f"Group #{gi}: {fmt}")
        print(f"Count: {len(members)}")

        example_shapes = [m[1].shape for m in members if isinstance(m[1], pd.DataFrame)]
        print(f"Example shapes (first 10): {example_shapes[:10]}")

        # Print a few examples per group
        for ex_i, (name, df) in enumerate(members[:max_dfs_per_group_to_print], start=1):
            if df is None or not isinstance(df, pd.DataFrame) or df.empty:
                print(f"  [Example {ex_i}] {name}: EMPTY/None")
                continue

            first_col = df.columns[0]
            vals = [v for v in df[first_col].head(20).tolist() if str(v).strip() and str(v).lower() != "nan"]
            vals = vals[:samples_per_df]

            print(f"  [Example {ex_i}] {name}")
            print(f"    first_col: {first_col!r}")
            print(f"    columns: {list(df.columns)[:12]}{' ...' if len(df.columns) > 12 else ''}")
            print(f"    first_col_samples: {vals}")

        print()


# Example usage with your list of performance tables
performances = []
perf_names = []
for i, fund in enumerate(funds_total):
    if fund.ticker in performance_funds and fund.performance_table is not None:
        performances.append(fund.performance_table)
        perf_names.append(f"{fund.ticker} | {fund.name} | {fund.share_class}")

describe_first_column_formats(performances, names=perf_names)

Total dataframes: 119
Unique first-column formats: 2

Group #1: MON_YY
Count: 118
Example shapes (first 10): [(62, 6), (62, 6), (62, 6), (62, 6), (107, 4), (120, 4), (120, 4), (67, 4), (120, 4), (120, 4)]
  [Example 1] EAOK | iShares ESG Aware 30/70 Conservative Allocation ETF | ETF Shares
    first_col: 'Unnamed: 0'
    columns: ['Unnamed: 0', 'Fund', 'Bloomberg U.S. Universal Index', 'MSCI All Country World Index (Net)', 'BlackRock ESG Aware Conservative Allocation Index', 'S&P Target Risk Conservative Index']
    first_col_samples: ['Jun 20', 'Jul 20', 'Aug 20']
  [Example 2] EAOM | iShares ESG Aware 40/60 Moderate Allocation ETF | ETF Shares
    first_col: 'Unnamed: 0'
    columns: ['Unnamed: 0', 'Fund', 'Bloomberg U.S. Universal Index', 'MSCI All Country World Index (Net)', 'BlackRock ESG Aware Moderate Allocation Index', 'S&P Target Risk Moderate Index']
    first_col_samples: ['Jun 20', 'Jul 20', 'Aug 20']
  [Example 3] EAOR | iShares ESG Aware 60/40 Balanced Allocation ETF | ET

In [ ]:
import sys
from pathlib import Path
%reload_ext autoreload
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


from src.simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        print(fund.performance_table)
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        print("---")

    Unnamed: 0     Fund Morningstar US Market Index  \
0       May 15  $10,181                     $10,137   
1       Jun 15  $10,069                      $9,955   
2       Jul 15  $10,492                     $10,133   
3       Aug 15   $9,796                      $9,524   
4       Sep 15   $9,536                      $9,256   
..         ...      ...                         ...   
115     Dec 24  $40,608                     $32,198   
116     Jan 25  $41,636                     $33,206   
117     Feb 25  $40,089                     $32,622   
118     Mar 25  $36,755                     $30,706   
119     Apr 25  $37,541                     $30,526   

    Morningstar® US Large-Mid Cap Broad Growth Index℠  
0                                             $10,183  
1                                             $10,074  
2                                             $10,500  
3                                              $9,803  
4                                              $9,545  
.. 

In [ ]:
tickers = []
unique_funds = []
duplicates = 0

for fund in funds_total:
    if fund.ticker in tickers:
        print(f"DUPLICATE: {fund.name} ({fund.ticker})")
        duplicates += 1
    else:
        tickers.append(fund.ticker)
        unique_funds.append(fund)

# Replace the original list
funds_total = unique_funds
print(f"Removed {duplicates} duplicates")
print(f"Remaining funds: {len(funds_total)}")

DUPLICATE: iShares 0-5 Year High Yield Corporate Bond ETF (SHYG)
DUPLICATE: iShares 0-5 Year Investment Grade Corporate Bond ETF (SLQD)
DUPLICATE: iShares 1-3 Year International Treasury Bond ETF (ISHG)
DUPLICATE: iShares 20+ Year Treasury Bond BuyWrite Strategy ETF (TLTW)
DUPLICATE: iShares Aaa - A Rated Corporate Bond ETF (QLTA)
DUPLICATE: iShares BB Rated Corporate Bond ETF (HYBB)
DUPLICATE: iShares Broad USD High Yield Corporate Bond ETF (USHY)
DUPLICATE: iShares CMBS ETF (CMBS)
DUPLICATE: iShares Convertible Bond ETF (ICVT)
DUPLICATE: iShares Core 1-5 Year USD Bond ETF (ISTB)
DUPLICATE: iShares Core International Aggregate Bond ETF (IAGG)
DUPLICATE: iShares ESG Advanced High Yield Corporate Bond ETF (HYXF)
DUPLICATE: iShares Fallen Angels USD Bond ETF (FALN)
DUPLICATE: iShares Floating Rate Bond ETF (FLOT)
DUPLICATE: iShares GNMA Bond ETF (GNMA)
DUPLICATE: iShares High Yield Corporate Bond BuyWrite Strategy ETF (HYGW)
DUPLICATE: iShares iBonds 2025 Term High Yield and Income ETF (

In [ ]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Successfully saved 420 funds to pickle file
File size: 3791.39 KB


In [4]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 420 funds from pickle file


## Summary Prospectus

In [60]:
from edgar import Company, set_identity
import pandas as pd
from typing import List, Dict
import sys
from tqdm import tqdm
from IPython.display import display, Markdown
from src.simple_rag.extraction.general_info import FundInfoExtractor
from pathlib import Path

set_identity('luis.alvarez.conde@alumnos.upm.es')
tickers = ["VOO", "MGK", "HEZU", "VMGRX", "VDIGX"]

for ticker in tickers:
    company = Company(ticker)
    processed_funds = []
    filings = company.get_filings(form="497K")

    for filing in filings:
        text = filing.text()
        extractor = FundInfoExtractor(text, ticker=ticker)
        fund_data = extractor.get_structured_data()
        if fund_data['ticker'] in processed_funds:
            print("First duplicate: ", fund_data['ticker'])
            break
        
        processed_funds.append(fund_data['ticker'])
        md = extractor.get_clean_markdown()
        for fund in funds_total:
            if fund.ticker == fund_data['ticker']:
                fund.summary_prospectus = md
                fund.managers = fund_data['managers']
                fund.strategies = fund_data['strategies']
                fund.risks = fund_data['risks']
                fund.objective = fund_data['objective']
                break
        
    print("Processed funds: ", len(processed_funds), "for ticker: ", ticker)


First duplicate:  VSCIX
Processed funds:  48 for ticker:  VOO


First duplicate:  VEXC
Processed funds:  36 for ticker:  MGK


First duplicate:  XT
Processed funds:  120 for ticker:  HEZU


First duplicate:  VMGRX
Processed funds:  1 for ticker:  VMGRX


First duplicate:  VGSNX
Processed funds:  11 for ticker:  VDIGX


In [61]:
managers = set()
for fund in funds_total:
    if fund.managers is not None:
        # If fund.managers is a list, add each manager individually
        if isinstance(fund.managers, list):
            managers.update(fund.managers)  # ✅ Use update() for lists
        else:
            managers.add(fund.managers)    # ✅ Use add() for single values

print(f"Unique managers found: {len(managers)}")
for manager in sorted(managers):
    print(f"  - {manager}")

Unique managers found: 32
  - Aaron Choi
  - Asian Economic Risk
  - Aurélie Denis
  - Brett Barakett
  - Chris Nieves
  - Christopher Chung
  - Consumer Goods
  - Erin Armstrong
  - Gary Robinson
  - Investment Manager
  - Jake Riley
  - James Mauro
  - Jena Stenger
  - Jennifer Hsui
  - Jonathan Graves
  - Kenny Narzikul
  - Lawrence Burns
  - Managing Director
  - Matt Waldron
  - Michael Cling
  - Michelle Louie
  - Nataliya Kofman
  - Natasha Kuhlkin
  - Nick Birkett
  - Peter Sietsema
  - Senior Managing Director
  - Simon Webber
  - Steven White
  - Suzanne Ly
  - Thomas Coutts
  - Tom Slater
  - Walter Nejman


In [62]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Successfully saved 420 funds to pickle file
File size: 50133.35 KB


In [ ]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd

from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/src")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))
import sys
from simple_rag.models.fund import FundData

PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
funds_total = []
with PKL_PATH.open("rb") as f:

    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

ModuleNotFoundError: No module named 'simple_rag'

## NPORT (Portfolio Composition)

In [3]:
from edgar import Company, set_identity
import sys

# 1. Set Identity
set_identity('luis.alvarez.conde@alumnos.upm.es')

def test_series_extraction(ticker):
    print(f"--- Testing Series ID Extraction for {ticker} ---")
    
    try:
        # 2. Get Filings
        company = Company(ticker)
        
        # .latest(1) returns the single object directly in your version, not a list
        filing = company.get_filings(form="NPORT-P").latest(1)
        
        if not filing:
            print(f"No NPORT-P filings found for {ticker}")
            return

        print(f"Accessing filing date: {filing.report_date}")
        
        # 3. Parse XML
        # This downloads the actual data
        xml_data = filing.obj()
        
        # 4. Extract Identifiers
        # The Series ID is usually located in the fund_series object
        fund_series = xml_data.get_fund_series()
        series_id = fund_series.series_id
        series_name = fund_series.name
        print(series_id)
        # Verify General Info
        gen_info = xml_data.general_info
        
        print("\n=== EXTRACTION RESULTS ===")
        print(f"Ticker:         {ticker}")
        print(f"CIK:            {filing.cik}")
        print(f"Series ID:      {series_id}  <-- THIS IS YOUR KEY")
        print(f"Series Name:    {series_name}")
        
        # Optional: Check Class IDs (The specific share classes)
        if hasattr(gen_info, 'series_classes'):
            print("\nAssociated Share Classes (Tickers in this Series):")
            for cls in gen_info.series_classes:
                print(f" - Class ID: {cls.class_id} | Ticker: {cls.ticker_symbol}")
                
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

# Run the test
test_series_extraction("VOO")

--- Testing Series ID Extraction for VOO ---
Accessing filing date: 2025-09-30
S000012757

=== EXTRACTION RESULTS ===
Ticker:         VOO
CIK:            36405
Series ID:      S000012757  <-- THIS IS YOUR KEY
Series Name:    VANGUARD MID-CAP VALUE INDEX FUND


In [ ]:
from edgar import Company, set_identity
import pandas as pd
from typing import List, Dict
import sys
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing as mp
from threading import Lock
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from src.simple_rag.extraction.nport import NPortProcessor
from src.simple_rag.models.fund import PortfolioHolding, Derivatives, NonDerivatives
from pathlib import Path

company_json_path = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json")

set_identity('luis.alvarez.conde@alumnos.upm.es')

def process_single_filing(filing, ticker, company_json_path):
    """Process a single filing - can be parallelized"""
    try:
        xml_data = filing.obj()
        fund_name = xml_data.get_fund_series().name
        reporting_period = xml_data.reporting_period
        portfolio_list = xml_data.investments
        derivatives = xml_data.derivatives
        series_id = xml_data.series_id

        # Process holdings
        proc = NPortProcessor(company_tickers_json_path=company_json_path, min_similarity=0.74)
        holdings = proc.process_holdings(portfolio_list)
        result = proc.enrich_tickers(holdings, verbose=False)  # Set verbose=False to reduce I/O
        
        not_matches = result[result['matched_ticker'].isna() | (result['matched_ticker'] == '')]
        
        return {
            'fund_name': fund_name,
            'reporting_period': reporting_period,
            'holdings': holdings,
            'result': result,
            'derivatives': derivatives,
            'not_matches': not_matches,
            'ticker': ticker,
            'series_id': series_id,
            'report_date': filing.report_date
        }
    except Exception as e:
        print(f"Error processing filing for {ticker}: {e}")
        return None

def process_ticker(ticker, company_json_path):
    """Process all filings for a single ticker - SEQUENTIAL within ticker"""
    try:
        nport_file = Company(ticker)
        filings = sorted(nport_file.get_filings(form="NPORT-P"), 
                        key=lambda x: x.report_date, reverse=True)
        
        if not filings:
            print(f"No filings found for {ticker}")
            return None
            
        print(f"Processing ticker: {ticker}, most recent filing date: {filings[0].report_date}")
        
        funds_processed_set = set()
        ticker_results = []
        
        # Process filings SEQUENTIALLY for this ticker (to respect the stop condition)
        for filing in filings:
            result = process_single_filing(filing, ticker, company_json_path)
            
            if result is not None:
                # Check if we've already processed this fund
                if result['fund_name'].lower() in funds_processed_set:
                    print(f"Stopping - already processed fund: {result['fund_name']}")
                    break
                
                funds_processed_set.add(result['fund_name'].lower())
                ticker_results.append(result)
                
                print(f"{ticker} - Fund: {result['fund_name']}, Holdings: {len(result['holdings'])}, Unmatched: {len(result['not_matches'])}")
        
        return {
            'ticker': ticker,
            'results': ticker_results,
            'funds_processed': list(funds_processed_set)
        }
    
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")
        return None

# Main execution - PARALLEL across tickers only
tickers = ["VOO", "MGK", "HEZU", "VMGRX", "VDIGX"]

# Use fewer workers to avoid overwhelming the system
max_workers = min(5, len(tickers))  # Start with 3 workers
print(f"Using {max_workers} workers for tickers")

all_results = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_ticker = {
        executor.submit(process_ticker, ticker, company_json_path): ticker 
        for ticker in tickers
    }
    
    for future in tqdm(as_completed(future_to_ticker), total=len(tickers), desc="Processing tickers"):
        ticker = future_to_ticker[future]
        try:
            result = future.result()
            if result:
                all_results.append(result)
                print(f"\nCompleted {ticker}: {len(result['funds_processed'])} funds processed")
        except Exception as e:
            print(f"Error with ticker {ticker}: {e}")

# Update funds_total object with the results
print("\n=== Updating funds_total ===")

processor = NPortProcessor()
for ticker_result in all_results:
    for filing_result in ticker_result['results']:
        ticker = filing_result['ticker']
        reporting_period = filing_result['reporting_period']
        holdings = filing_result['holdings']
        derivatives = filing_result['derivatives']
        series_id = filing_result['series_id']
        # Update your funds_total structure
        for fund in funds_total:
            if ticker == fund.ticker():
                print(f"Updating fund: {fund.name}")
                df = processor.to_df(holdings)

                fund.non_derivatives = NonDerivatives(
                    date=reporting_period,
                    holdings_df=df
                )
                df2 = processor.to_df(derivatives)
                fund.derivatives = Derivatives(
                    date=reporting_period,
                    derivatives_df=df2
                )
                fund.series_id = series_id
                break

print("\n=== Processing Complete ===")
print(f"Total tickers processed: {len(all_results)}")
for result in all_results:
    print(f"{result['ticker']}: {len(result['funds_processed'])} funds")



def verify_fund_data_integrity(funds_list):
    """
    Iterates through funds to verify that the holdings DataFrame is populated 
    and counts how many are None or missing.
    """
    print("\n" + "="*40)
    print("DATA INTEGRITY VERIFICATION")
    print("="*40)
    
    none_count = 0
    valid_count = 0
    
    for fund in funds_list:
        # We need to check layers: 
        # 1. Does fund.non_derivatives exist? 
        # 2. Is it not None?
        # 3. Is the holdings_df inside it not None?
        
        has_data = False
        row_count = 0
        
        try:
            if (hasattr(fund, 'non_derivatives') and 
                fund.non_derivatives is not None and 
                fund.non_derivatives.holdings_df is not None):
                
                has_data = True
                row_count = len(fund.non_derivatives.holdings_df)
        except Exception:
            # If any attribute access fails, treat as no data
            has_data = False

        if has_data:
            valid_count += 1
            # Optional: Print success if you want to see the good ones
            # print(f"✓ {fund.name:<20} | Rows: {row_count}")
        else:
            none_count += 1
            print(f"❌ {fund.name:<20} | Status: DATAFRAME IS NONE/MISSING")

    print("-" * 40)
    print(f"Total Funds Checked: {len(funds_list)}")
    print(f"Valid DataFrames:    {valid_count}")
    print(f"None/Missing Values: {none_count}")
    print("=" * 40)

# Run the verification
# Note: Ensure 'funds_total' is defined in your scope (it was referenced in your loop)
if 'funds_total' in locals():
    verify_fund_data_integrity(funds_total)
else:
    print("Error: 'funds_total' variable not found in current scope.")

Using 5 workers for tickers


Processing tickers:   0%|          | 0/5 [00:00<?, ?it/s]

Processing ticker: VOO, most recent filing date: 2025-09-30
Error processing filing for VOO: 'FundReport' object has no attribute 'series_id'
Processing ticker: MGK, most recent filing date: 2025-09-30
Error processing filing for MGK: 'FundReport' object has no attribute 'series_id'
Processing ticker: VMGRX, most recent filing date: 2025-10-31
Processing ticker: VDIGX, most recent filing date: 2025-10-31
Error processing filing for VMGRX: 'FundReport' object has no attribute 'series_id'
Error processing filing for VDIGX: 'FundReport' object has no attribute 'series_id'
Error processing filing for VDIGX: 'FundReport' object has no attribute 'series_id'
Processing ticker: HEZU, most recent filing date: 2025-10-31
Error processing filing for VDIGX: 'FundReport' object has no attribute 'series_id'
Error processing filing for MGK: 'FundReport' object has no attribute 'series_id'
Error processing filing for VDIGX: 'FundReport' object has no attribute 'series_id'
Error processing filing for M

Processing tickers:  20%|██        | 1/5 [10:02<40:10, 602.60s/it]

Error processing filing for VDIGX: 'FundReport' object has no attribute 'series_id'

Completed VDIGX: 0 funds processed
Error processing filing for VMGRX: 'FundReport' object has no attribute 'series_id'
Error processing filing for VOO: 'FundReport' object has no attribute 'series_id'
Error processing filing for VMGRX: 'FundReport' object has no attribute 'series_id'
Error processing filing for MGK: 'FundReport' object has no attribute 'series_id'
Error processing filing for MGK: 'FundReport' object has no attribute 'series_id'
Error processing filing for VMGRX: 'FundReport' object has no attribute 'series_id'
Error processing filing for MGK: 'FundReport' object has no attribute 'series_id'
Error processing filing for VMGRX: 'FundReport' object has no attribute 'series_id'
Error processing filing for VMGRX: 'FundReport' object has no attribute 'series_id'
Error processing filing for MGK: 'FundReport' object has no attribute 'series_id'
Error processing filing for VOO: 'FundReport' obje

In [65]:
import pickle
from pathlib import Path

PKL_PATH = Path("./funds_backup.pkl")
TMP_PATH = PKL_PATH.with_suffix(PKL_PATH.suffix + ".tmp")

with TMP_PATH.open("wb") as f:
    pickle.dump(funds_total, f, protocol=pickle.HIGHEST_PROTOCOL)

TMP_PATH.replace(PKL_PATH)

print(f"Saved {len(funds_total)} funds to pickle file: {PKL_PATH.resolve()}")
print(f"File size: {PKL_PATH.stat().st_size / (1024 * 1024):.2f} MB")

Saved 420 funds to pickle file: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
File size: 48.96 MB


In [10]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/src")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 420 funds from pickle file


## Processing Phase

### General information about the fund

In [30]:

for fund in funds_total:
    if 'Vanguard' in fund.name:
        fund.provider = 'The Vanguard Group, Inc'
    else:
        fund.provider = 'BlackRock, Inc'

In [31]:
registrants = set()
for fund in funds_total:
    if '\xa0' in fund.registrant:
        fund.registrant = fund.registrant.replace('\xa0', ' ')
    registrants.add(fund.registrant)

print(registrants)

{'Vanguard Specialized Funds', 'Vanguard Index Funds', 'Vanguard World Fund', 'iShares Trust', 'Vanguard Whitehall Funds'}


In [32]:
from src.simple_rag.models.fund import ShareClassType

unique_share_classes = set(fund.share_class for fund in funds_total)
print("Unique Share Classes:")
for share_class in unique_share_classes:
    print(share_class)

from collections import Counter

# Count funds by share class
share_counts = Counter()
for fund in funds_total:
    if fund.share_class:
        if fund.share_class == ShareClassType.OTHER:
            fund.share_class = ShareClassType.ETF
        share_counts[fund.share_class] += 1

print("Funds by share class:")
for share_type, count in share_counts.items():
    print(f"  {share_type.value}: {count} funds")



Unique Share Classes:
ShareClassType.ETF
ShareClassType.INVESTOR
ShareClassType.ADMIRAL
ShareClassType.OTHER
ShareClassType.INSTITUTIONAL_SELECT
ShareClassType.INSTITUTIONAL_PLUS
ShareClassType.INSTITUTIONAL
Funds by share class:
  Investor Shares: 27 funds
  ETF Shares: 330 funds
  Admiral Shares: 40 funds
  Institutional Shares: 16 funds
  Institutional Plus Shares: 4 funds
  Institutional Select Shares: 3 funds


In [33]:
from collections import Counter
import unicodedata

def normalize_name(text):
    if not text:
        return "Unknown"
    # NFKD normalization turns non-breaking spaces into normal spaces
    return unicodedata.normalize("NFKD", text).strip()

# Your original loop, but with the fix applied:
share_counts = Counter()

for fund in funds_total:
    if fund.registrant:
        # Apply the fix here
        clean_registrant = normalize_name(fund.registrant)
        share_counts[clean_registrant] += 1

print("Funds by share class (Merged):")
for share_type, count in share_counts.items():
    print(f"  {share_type}: {count} funds")

Funds by share class (Merged):
  Vanguard Index Funds: 52 funds
  Vanguard World Fund: 41 funds
  Vanguard Specialized Funds: 14 funds
  Vanguard Whitehall Funds: 18 funds
  iShares Trust: 295 funds


In [34]:
from collections import Counter
import unicodedata

def normalize_name(text):
    if not text:
        return "Unknown"
    # NFKD normalization turns non-breaking spaces into normal spaces
    return unicodedata.normalize("NFKD", text).strip()

fund_counts = Counter()

for fund in funds_total:
    if fund.name:
        # 1. Normalize the name first
        clean_name = normalize_name(fund.name)
        
        # 2. Check if it starts with iShares or Vanguard
        # We check case-sensitively, but you could use .lower() if needed
        if not (clean_name.startswith("iShares") or clean_name.startswith("Vanguard")):
            # 3. Add 'Vanguard' if missing
            clean_name = f"Vanguard {clean_name}"
            
        # 4. Count the final modified name
        fund_counts[clean_name] += 1

print("Funds by name (Merged & Fixed):")
for fund_name, count in fund_counts.items():
    print(f"  {fund_name}: {count} funds")



Funds by name (Merged & Fixed):
  Vanguard Extended Market Index Fund: 6 funds
  Vanguard Mid-Cap Index Fund: 5 funds
  Vanguard Mid-Cap Growth Index Fund: 3 funds
  Vanguard Mid-Cap Value Index Fund: 3 funds
  Vanguard Small-Cap Index Fund: 5 funds
  Vanguard Small-Cap Growth Index Fund: 4 funds
  Vanguard Small-Cap Value Index Fund: 4 funds
  Vanguard Total Stock Market Index Fund: 6 funds
  Vanguard 500 Index Fund: 4 funds
  Vanguard Value Index Fund: 4 funds
  Vanguard Growth Index Fund: 4 funds
  Vanguard Large-Cap Index Fund: 4 funds
  Vanguard Mega Cap Growth Index Fund: 2 funds
  Vanguard Extended Duration Treasury Index Fund: 2 funds
  Vanguard ESG U.S. Stock ETF: 1 funds
  Vanguard ESG International Stock ETF: 1 funds
  Vanguard Global WellingtonTM Fund: 2 funds
  Vanguard Global Wellesley® Income Fund: 2 funds
  Vanguard ESG U.S. Corporate Bond ETF: 1 funds
  Vanguard FTSE Social Index Fund: 2 funds
  Vanguard Materials Index Fund: 2 funds
  Vanguard Communication Services I

In [35]:
sum = 0
for fund in funds_total:
    # Format correctly the numeric fields
    fund.net_assets *= 1e6
    fund.advisory_fees *= 1e3
   

In [36]:
from src.simple_rag.utils.chart_utils import extract_flexible_performance
from src.simple_rag.models.fund import AverageReturnSnapshot
for fund in funds_total:
    
    year = fund.report_date.year
    if fund.avg_annual_returns is None:
        continue 
    result = extract_flexible_performance(fund.avg_annual_returns)
    
    # Create AverageReturnSnapshot instance
    snapshot = AverageReturnSnapshot(
        return_1y=result.get('1_year', None),
        return_5y=result.get('5_year', None),
        return_10y=result.get('10_year', None),
        return_inception=result.get('since_inception', None)
    )
    
    fund.performance[str(year)] = snapshot 
    

In [37]:
for fund in funds_total:
    if fund.financial_highlights is not None:
        print(fund.financial_highlights)
        for _, values in fund.financial_highlights.items():
            values.net_assets = values.net_assets * 1e6

{'2024': FinancialHighlights(turnover=11.0, expense_ratio=0.19, total_return=16.76, net_assets=195.0, net_assets_value_begining=124.78, net_assets_value_end=144.2, net_income_ratio=1.09), '2023': FinancialHighlights(turnover=11.0, expense_ratio=0.19, total_return=25.22, net_assets=232.0, net_assets_value_begining=100.93, net_assets_value_end=124.78, net_income_ratio=1.28), '2022': FinancialHighlights(turnover=11.0, expense_ratio=0.19, total_return=-26.56, net_assets=229.0, net_assets_value_begining=138.8, net_assets_value_end=100.93, net_income_ratio=1.14), '2021': FinancialHighlights(turnover=19.0, expense_ratio=0.19, total_return=12.31, net_assets=399.0, net_assets_value_begining=124.83, net_assets_value_end=138.8, net_income_ratio=0.87), '2020': FinancialHighlights(turnover=19.0, expense_ratio=0.19, total_return=32.04, net_assets=454.0, net_assets_value_begining=95.66, net_assets_value_end=124.83, net_income_ratio=1.04)}
{'2024': FinancialHighlights(turnover=11.0, expense_ratio=0.06

In [38]:
managers_set = set()
for fund in funds_total:
    if hasattr(fund, 'managers') and fund.managers is not None:
        managers_set.update(fund.managers)
for manager in sorted(managers_set):
    print(manager)

Aaron Choi
Asian Economic Risk
Aurélie Denis
Brett Barakett
Chris Nieves
Christopher Chung
Consumer Goods
Erin Armstrong
Gary Robinson
Investment Manager
Jake Riley
James Mauro
Jena Stenger
Jennifer Hsui
Jonathan Graves
Kenny Narzikul
Lawrence Burns
Managing Director
Matt Waldron
Michael Cling
Michelle Louie
Nataliya Kofman
Natasha Kuhlkin
Nick Birkett
Peter Sietsema
Senior Managing Director
Simon Webber
Steven White
Suzanne Ly
Thomas Coutts
Tom Slater
Walter Nejman


Enhance the summary prospectus 

In [63]:
def update_summary_prospectus_with_header(fund):
    """Remove all existing headers and add new header."""
    
    header = f"# FUND PROFILE({fund.ticker} - {fund.name} - {fund.share_class if fund.share_class else 'N/A'}):"
    
    if not fund.summary_prospectus or not fund.summary_prospectus.strip():
        return header
    
    # Remove all header lines and keep content
    lines = fund.summary_prospectus.split('\n')
    content_lines = [line for line in lines if not line.strip().startswith("# FUND PROFILE(")]
    
    # Add new header at beginning with proper spacing
    if content_lines:
        return f"{header}\n" + '\n'.join(content_lines)
    else:
        return header

# Update all funds
for i, fund in enumerate(funds_total):
    fund.summary_prospectus = update_summary_prospectus_with_header(fund)
    print(f"✅ {i+1}/{len(funds_total)}: Updated {fund.ticker}")

✅ 1/420: Updated VEXMX
✅ 2/420: Updated VXF
✅ 3/420: Updated VEXAX
✅ 4/420: Updated VIEIX
✅ 5/420: Updated VEMPX
✅ 6/420: Updated VSEMX
✅ 7/420: Updated VIMSX
✅ 8/420: Updated VO
✅ 9/420: Updated VIMAX
✅ 10/420: Updated VMCIX
✅ 11/420: Updated VMCPX
✅ 12/420: Updated VMGIX
✅ 13/420: Updated VOT
✅ 14/420: Updated VMGMX
✅ 15/420: Updated VMVIX
✅ 16/420: Updated VOE
✅ 17/420: Updated VMVAX
✅ 18/420: Updated NAESX
✅ 19/420: Updated VB
✅ 20/420: Updated VSMAX
✅ 21/420: Updated VSCIX
✅ 22/420: Updated VSCPX
✅ 23/420: Updated VISGX
✅ 24/420: Updated VBK
✅ 25/420: Updated VSGAX
✅ 26/420: Updated VSGIX
✅ 27/420: Updated VISVX
✅ 28/420: Updated VBR
✅ 29/420: Updated VSIAX
✅ 30/420: Updated VSIIX
✅ 31/420: Updated VTSMX
✅ 32/420: Updated VTI
✅ 33/420: Updated VTSAX
✅ 34/420: Updated VITSX
✅ 35/420: Updated VSMPX
✅ 36/420: Updated VSTSX
✅ 37/420: Updated VFINX
✅ 38/420: Updated VOO
✅ 39/420: Updated VFIAX
✅ 40/420: Updated VFFSX
✅ 41/420: Updated VIVAX
✅ 42/420: Updated VTV
✅ 43/420: Updated VVIAX

In [64]:
# Check a few funds to see the result
for fund in funds_total[:3]:
    print(f"\n{fund.ticker}:")
    print(f"First 100 chars: ...")
    print(fund.summary_prospectus)
    print(f"Has header: {'# FUND PROFILE(' in fund.summary_prospectus}")


VEXMX:
First 100 chars: ...
# FUND PROFILE(VEXMX - Vanguard Extended Market Index Fund - Investor Shares):

## Investment Objective
The Fund seeks to track the performance of a benchmark index that measuresthe investment return of small- and mid-capitalization stocks.

## Principal Investment Strategies
The Fund employs an indexing investment approach designed to track theperformance of the S& P Completion Index (the Index), a broadly diversified indexof stocks of small and mid-size U. S. companies. The Index contains all of theU. S. common stocks regularly traded on the New York Stock Exchange, Cboe, and the Nasdaq over-the-counter market, except those stocks included in theS& P 500 Index. The Fund invests by sampling the Index, meaning that it holds abroadly diversified collection of securities that, in the aggregate, approximatesthe full Index in terms of key characteristics. These characteristics includeindustry weightings and market capitalization, as well as certain financialmea

### Geographic Allocation

In [11]:

for fund in funds_total:
    df = fund.geographic_allocation
    if df is not None:
        df = df.iloc[:,[0,1]]
        df.columns = ['Country', 'Percentage']
        df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.replace('\u200b', '', regex=True).str.strip()
        df.iloc[:, 0] = df.iloc[:, 0].replace(['', 'nan', 'None', 'Other#', 'Country/Geographic Region'], 'Other')
        df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce')
        df = df[df.iloc[:, 1].notna()]
        
        fund.geographic_allocation = df
        
countries = set()
for fund in funds_total:
    df = fund.geographic_allocation
    if df is not None:
        # Get unique values from the first column
        unique_values = df.iloc[:, 0].unique()
        countries.update(unique_values)
print(f"Unique countries: {len(countries)}")
print("Countries:", sorted(countries))
# Verify data types
for fund in funds_total:  # Check first fund
    if fund.geographic_allocation is not None:
        print(f"\nSample cleaned data for {fund.ticker}:")
        print(f"Shape: {fund.geographic_allocation.shape}")
        print(f"Data types: {fund.geographic_allocation.dtypes}")
        print(f"Sample rows:")
        print(fund.geographic_allocation)
        break


Unique countries: 42
Countries: ['Australia', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'Colombia', 'Denmark', 'Finland', 'France', 'Germany', 'Hong Kong', 'India', 'Indonesia', 'Israel', 'Italy', 'Japan', 'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Other', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Saudi Arabia', 'Singapore', 'South Africa', 'South Korea', 'Spain', 'Supranational', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'United Arab Emirates', 'United Kingdom', 'United States']

Sample cleaned data for ERET:
Shape: (10, 2)
Data types: Country       object
Percentage    object
dtype: object
Sample rows:
           Country Percentage
1    United States       62.5
2            Japan        9.7
3        Australia        5.1
4   United Kingdom        4.4
5        Singapore        3.0
6        Hong Kong        2.7
7           Sweden        2.2
8          Germany        2.2
9           France        2.0
10          Canada        2.0


### Sector Allocation


In [12]:
from src.simple_rag.utils.chart_utils import validate_and_clean_allocation

for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        df = validate_and_clean_allocation(df, 'Sector', sort_by_value=True)
        fund.sector_allocation = df


VALIDATION REPORT
Category Column: 'Sector' | Value Column: 'Percent of Total Investments(a)'
❌ Row  0: 'Portfolio Composition % of Net' | 'nan     ' | Numeric: True | Range OK: False
✅ Row  1: 'Communication Services        ' | '4.3%    ' | Numeric: True | Range OK: True
✅ Row  2: 'Consumer Discretionary        ' | '12.0%   ' | Numeric: True | Range OK: True
✅ Row  3: 'Consumer Staples              ' | '3.0%    ' | Numeric: True | Range OK: True
✅ Row  4: 'Energy                        ' | '4.1%    ' | Numeric: True | Range OK: True
✅ Row  5: 'Financials                    ' | '18.0%   ' | Numeric: True | Range OK: True
✅ Row  6: 'Health Care                   ' | '11.4%   ' | Numeric: True | Range OK: True
✅ Row  7: 'Industrials                   ' | '17.4%   ' | Numeric: True | Range OK: True
✅ Row  8: 'Information Technology        ' | '17.9%   ' | Numeric: True | Range OK: True
✅ Row  9: 'Materials                     ' | '4.7%    ' | Numeric: True | Range OK: True
✅ Row 10: 'Real

In [13]:
import pandas as pd
import re

# 1. Get all sectors from your funds
sectors = set()
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        sectors.update(df.iloc[:, 0].unique())

# 2. The Cleaning Function
def get_standard_key(text):
    if not isinstance(text, str):
        return text
        
    # A. Convert to lowercase
    clean = text.lower()
    
    # B. Remove the specific word "sector"
    clean = re.sub(r'\bsector\b', '', clean)
    
    # C. Remove punctuation
    clean = re.sub(r'[^a-z0-9\s]', '', clean)
    
    # D. Remove extra whitespace
    clean = re.sub(r'\s+', ' ', clean).strip()
    
    return clean

# 3. Create mapping DataFrame
sectors_list = list(sectors)
df_mapping = pd.DataFrame({'Original': sectors_list})
df_mapping['Clean_Key'] = df_mapping['Original'].apply(get_standard_key)

# 4. Create canonical mapping (pick shortest name as standard)
canonical_map = df_mapping.groupby('Clean_Key')['Original'].transform(lambda x: sorted(x, key=len)[0])
df_mapping['Standardized'] = canonical_map

# 5. Create the mapping dictionary
standardization_dict = dict(zip(df_mapping['Original'], df_mapping['Standardized']))

# 6. Review the mapping
print("Standardization Mapping:")
print(df_mapping[['Original', 'Standardized']].drop_duplicates().sort_values('Standardized'))

# 7. Apply standardization to all fund DataFrames
updated_funds = 0
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        # Make a copy to avoid SettingWithCopyWarning
        df_copy = df.copy()
        
        # Apply standardization to first column
        original_values = df_copy.iloc[:, 0].copy()
        standardized_values = original_values.map(standardization_dict)
        df_copy.iloc[:, 0] = standardized_values
        df_copy.iloc[:, 0] = (
            df_copy.iloc[:, 0]
            .str.replace(r'\bSector\b', '', regex=True)  # Remove multiple words
            .str.replace(r'\s+', ' ', regex=True)         # Fix extra spaces
            .str.strip()                                 # Remove leading/trailing spaces
        )
        # Update the fund's DataFrame
        fund.sector_allocation = df_copy
        updated_funds += 1

print(f"\n✅ Updated {updated_funds} funds with standardized sector names")

# 8. Verify the results
final_sectors = set()
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        final_sectors.update(df.iloc[:, 0].unique())

print(f"\nBefore standardization: {len(sectors)} unique sectors")
print(f"After standardization: {len(final_sectors)} unique sectors")
print(f"Reduction: {len(sectors) - len(final_sectors)} duplicates removed")

# 9. Show some examples of the changes
print(f"\nExamples of standardization:")
for original, standardized in standardization_dict.items():
    if original != standardized:
        print(f"  '{original}' → '{standardized}'")

Standardization Mapping:
                                Original                         Standardized
140                  Aerospace & Defense                  Aerospace & Defense
136             Aerospace Defense Sector                  Aerospace & Defense
70                                Africa                               Africa
97    Agricultural Farm Machinery Sector   Agricultural Farm Machinery Sector
117         Air Freight Logistics Sector              Air Freight & Logistics
..                                   ...                                  ...
141                            Utilities                            Utilities
67   Vanguard Real Estate II Index Fund1  Vanguard Real Estate II Index Fund1
124                      Water Utilities                      Water Utilities
64                Water Utilities Sector                      Water Utilities
111  Wireless Telecommunication Services  Wireless Telecommunication Services

[161 rows x 2 columns]

✅ Updated 249 

In [62]:
# Get unique values from first column (index 0) of all sector allocation DataFrames
sectors = set()
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        # Get unique values from first column
        unique_values = df.iloc[:, 0].unique()
        sectors.update(unique_values)

print(f"Unique sectors: {len(sectors)}")
print("Sectors:", sorted(sectors))

Unique sectors: 120
Sectors: ['Aerospace & Defense', 'Africa', 'Agricultural Farm Machinery', 'Air Freight & Logistics', 'Asia', 'Automobile Components', 'Automobile Manufacturers', 'Automobiles', 'Banks', 'Basic Materials', 'Beverages', 'Biotechnology', 'Broadline Retail', 'Building Products', 'Capital Markets', 'Chemicals', 'Coal & Consumable Fuels', 'Commercial Services & Supplies', 'Communication Services', 'Communications', 'Communications Equipment', 'Construction & Engineering', 'Construction Machinery Heavy Transportation Equipment', 'Construction Materials', 'Consumer Discretionary', 'Consumer Finance', 'Consumer Staples', 'Consumer Staples Distribution & Retail', 'Containers & Packaging', 'Data Center REITs', 'Distributors', 'Diversified Consumer Services', 'Diversified Telecommunication Services', 'Domestic Equity', 'Domestic Fixed Income', 'Electric Utilities', 'Electrical Components Equipment', 'Electrical Equipment', 'Electronic Components', 'Electronic Equipment, Instrum

### Industry Allocation

In [64]:
from src.simple_rag.utils.chart_utils import validate_and_clean_allocation

for fund in funds_total:
    df = fund.industry_allocation
    if df is not None:
        df = validate_and_clean_allocation(df, 'Industry', sort_by_value=True)
        fund.industry_allocation = df


🔍 Detected header row at index 0
New columns: ['Industry', 'Percent of Total  Investments(a)']
VALIDATION REPORT
Category Column: 'Industry' | Value Column: 'Percent of Total  Investments(a)'
✅ Row  0: 'Retail REITs                  ' | '18.0    ' | Numeric: True | Range OK: True
✅ Row  1: 'Industrial REITs              ' | '13.0    ' | Numeric: True | Range OK: True
✅ Row  2: 'Health Care REITs             ' | '9.9     ' | Numeric: True | Range OK: True
✅ Row  3: 'Multi-Family Residential REITs' | '9.4     ' | Numeric: True | Range OK: True
✅ Row  4: 'Real Estate Operating Companie' | '8.0     ' | Numeric: True | Range OK: True
✅ Row  5: 'Office REITs                  ' | '6.7     ' | Numeric: True | Range OK: True
✅ Row  6: 'Data Center REITs             ' | '6.6     ' | Numeric: True | Range OK: True
✅ Row  7: 'Diversified REITs             ' | '6.5     ' | Numeric: True | Range OK: True
✅ Row  8: 'Self Storage REITs            ' | '6.3     ' | Numeric: True | Range OK: True
✅ Row  

In [65]:
# Get unique values from first column (index 0) of all sector allocation DataFrames
sectors = set()
for fund in funds_total:
    df = fund.industry_allocation
    if df is not None:
        # Get unique values from first column
        unique_values = df.iloc[:, 0].unique()
        sectors.update(unique_values)

print(f"Unique sectors: {len(sectors)}")
print("Sectors:", sorted(sectors))

Unique sectors: 128
Sectors: ['Aerospace & Defense', 'Alternative Carriers', 'Automobile Components', 'Automobiles', 'Automobiles & Components', 'Banks', 'Biotechnology', 'Brewers', 'Building Products', 'Cable & Satellite', 'Capital Goods', 'Capital Markets', 'Chemicals', 'Coal & Consumable Fuels', 'Commercial & Professional Services', 'Commodity Chemicals', 'Communications Equipment', 'Construction & Engineering', 'Construction Machinery & Heavy Transportation Equipment', 'Construction Materials', 'Consumer Discretionary Distribution & Retail', 'Consumer Durables & Apparel', 'Consumer Finance', 'Consumer Services', 'Consumer Staples Merchandise Retail', 'Containers & Packaging', 'Copper', 'Data Center REITs', 'Distillers & Vintners', 'Diversified Banks', 'Diversified Metals & Mining', 'Diversified REITs', 'Diversified Real Estate Activities', 'Diversified Telecommunication Services', 'Electric Utilities', 'Electrical Components & Equipment', 'Electrical Equipment', 'Electronic Equipme

In [14]:
import pickle
from pathlib import Path

PKL_PATH = Path("./funds_backup.pkl")
TMP_PATH = PKL_PATH.with_suffix(PKL_PATH.suffix + ".tmp")

with TMP_PATH.open("wb") as f:
    pickle.dump(funds_total, f, protocol=pickle.HIGHEST_PROTOCOL)

TMP_PATH.replace(PKL_PATH)

print(f"Saved {len(funds_total)} funds to pickle file: {PKL_PATH.resolve()}")
print(f"File size: {PKL_PATH.stat().st_size / (1024 * 1024):.2f} MB")

Saved 420 funds to pickle file: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
File size: 74.73 MB
